In [2]:
!python --version

# cuda version
!nvidia-smi
!nvcc --version

# ubuntu version
!cat /etc/issue

# pytorch version
import torch
print(torch.__version__)

# # tensorflow version
# import tensorflow as tf
# print(tf.__version__)

Python 3.7.10
Sat Apr 10 00:54:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 455.23.05    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  TITAN V             On   | 00000000:19:00.0 Off |                  N/A |
| 29%   43C    P8    24W / 250W |      1MiB / 12066MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  TITAN V             On   | 00000000:1A:00.0 Off |                  

In [3]:
# # !pip install ../input/my-libs/timm-0.3.4-py3-none-any.whl
# !pip install ../input/my-libs/timm-0.4.5-py3-none-any.whl
# # !pip install ../input/my-libs/Pillow-8.0.1-cp37-cp37m-manylinux1_x86_64.whl
# # !pip install ../input/my-libs/requests-2.25.0-py2.py3-none-any.whl
# !pip install ../input/my-libs/cupy_cuda102-8.6.0-cp37-cp37m-manylinux1_x86_64.whl
# # !pip install ../input/my-libs/tokenizers-0.10.1-cp37-cp37m-manylinux1_x86_64.whl
# # !pip install ../input/my-libs/transformers-4.4.2-py3-none-any.whl

# # !pip install ../input/my-libs/Keras_Applications-1.0.8-py3-none-any.whl
# # !pip install ../input/my-libs/efficientnet-1.1.0-py3-none-any.whl


Processing /home/ace19/my-repo/Shopee-Product-Matching/input/my-libs/timm-0.4.5-py3-none-any.whl
  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 17.4MB 108kB/s eta 0:00:01    51% |████████████████▋               | 9.0MB 12.0MB/s eta 0:00:01
    15% |�^C██                           | 128.0MB 12.3MB/s eta 0:00:55 0% |                                | 593kB 10.6MB/s eta 0:01:16    3% |█▏                              | 29.8MB 9.8MB/s eta 0:01:20    6% |██                              | 51.0MB 10.7MB/s eta 0:01:11    7% |██▌                             | 62.8MB 11.1MB/s eta 0:01:07    8% |██▋                             | 65.8MB 11.6MB/s eta 0:01:04    9% |███                             | 73.6MB 12.2MB/s eta 0:01:01    9% |███                             | 74.2MB 11.2MB/s eta 0:01:06    9% |███                             | 77.2MB 12.2MB/s eta 0:01:00    15% |█████                           | 125.5MB 9.1MB/s eta 0:01:15

Operation cancelled b

In [6]:
# https://www.kaggle.com/cdeotte/rapids

import sys
!cp ../input/rapids/rapids.0.18.0 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
sys.path = ["/opt/conda/envs/rapids/lib/python3.7/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.7"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path 
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

cp: cannot create regular file '/opt/conda/envs/rapids.tar.gz': Permission denied
tar (child): rapids.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
cp: cannot stat '/opt/conda/envs/rapids/lib/libxgboost.so': No such file or directory


In [ ]:
import numpy as np, pandas as pd, gc
import cv2, matplotlib.pyplot as plt
import cudf, cuml, cupy
from cuml.feature_extraction.text import TfidfVectorizer, CountVectorizer
from cuml.neighbors import NearestNeighbors

print('RAPIDS',cuml.__version__)

In [ ]:
train = pd.read_csv('../input/shopee-product-matching/train.csv')
tmp = train.groupby('label_group').posting_id.agg('unique').to_dict()
train['target'] = train.label_group.map(tmp)
print('train shape is', train.shape )
train.head()

In [ ]:
test = pd.read_csv('../input/shopee-product-matching/test.csv')
test_gf = cudf.read_csv('../input/shopee-product-matching/test.csv')
print('Test shape is', test_gf.shape )
test_gf.head()

# **Text Embeddings**

In [ ]:
import string
from tqdm import tqdm


def removePunctuation(text):
    punc_translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    return text.translate(punc_translator)


test['title_clean'] = test['title'].apply(removePunctuation)
title_to_use = cudf.DataFrame(test).title_clean

print('Computing text embeddings...')
tfidf_vec = TfidfVectorizer(stop_words='english', 
                            binary=True, 
                            lowercase=True,
#                             max_df = 0.5,
#                             min_df = 2,
                            max_features=25000)
text_embeddings = tfidf_vec.fit_transform(title_to_use).toarray().astype(np.float32)
# text_embeddings = tfidf_vec.fit_transform(title_to_use)
print('text embeddings shape',text_embeddings.shape)

In [ ]:
# TODO: fix

# from cudf import Series
# from cuml.common.sparsefuncs import csr_row_normalize_l2


# # preds = []

# def efficient_csr_cosine_similarity(query, tfidf_matrix, matrix_normalized=False):
#     query = csr_row_normalize_l2(query, inplace=False)
#     if not matrix_normalized:
#         tfidf_matrix = csr_row_normalize_l2(tfidf_matrix, inplace=False)
    
#     return tfidf_matrix.dot(query.T)


# def document_search(text_df, query, vectorizer, tfidf_matrix, top_n=3):
#     query_vec = vectorizer.transform(query)
#     similarities = efficient_csr_cosine_similarity(query_vec, tfidf_matrix, matrix_normalized=True)
#     similarities = similarities.todense().reshape(-1)
#     best_idx = similarities.argsort()[-top_n:][::-1]
#     o = test.iloc[cupy.asnumpy(best_idx)].posting_id.values
    
#     return o.tolist()
    
# #     pp = cudf.DataFrame({
# #         'text': text_df['text'].iloc[best_idx],
# #         'similarity': similarities[best_idx]
# #     })
# #     return pp

# _knn = 50
# if text_embeddings.shape[0] <= 3:
#     _knn = 3
# else:
#     _knn = 50
# preds = document_search(test, title_to_use, tfidf_vec, text_embeddings, top_n=_knn)

In [ ]:
preds = []
CHUNK = 1024

print('Finding similar titles...')
CTS = len(test) // CHUNK
if len(test) % CHUNK != 0: 
    CTS += 1

for j in range( CTS ):
    
    a = j*CHUNK
    b = (j+1)*CHUNK
    b = min(b,len(test))
    print('chunk',a,'to',b)
    
    # COSINE SIMILARITY DISTANCE
    cts = cupy.matmul(text_embeddings, text_embeddings[a:b].T).T
    
    for k in range(b-a):
        print('cts[k,]: ', cts[k,])
        IDX = cupy.where(cts[k,]>0.7)[0]
        print('IDX: ', IDX)
        o = test.iloc[cupy.asnumpy(IDX)].posting_id.values
        preds.append(o.tolist())
        
del tfidf_vec, text_embeddings
_ = gc.collect()

In [ ]:
# if len(test) > 3:
#     KNN = 50
# else : 
#     KNN = 3

# model = NearestNeighbors(n_neighbors = KNN)
# model.fit(text_embeddings)
# distances, indices = model.kneighbors(text_embeddings)
# print('distances: ', distances)
# print('indices: ', indices)

# preds = []
# for k in tqdm(range(text_embeddings.shape[0])):
#     idx = np.where(distances[k,] < 0.8)[0]
#     ids = indices[k,idx]
#     posting_ids = test['posting_id'].iloc[ids].values
#     preds.append(posting_ids)

# del model, distances, indices
# gc.collect()

In [ ]:
test['text_embeddings'] = preds
test.head()

# **Text Embeddings2**

In [ ]:
# import math
# import tensorflow as tf
# import efficientnet.tfkeras as efn
# from tqdm.notebook import tqdm
# from cuml.neighbors import NearestNeighbors
# from shutil import copyfile
# copyfile(src = "../input/shopee-external-models/tokenization.py", dst = "../working/tokenization.py")
# import tokenization
# import tensorflow_hub as hub


# # Arcmarginproduct class keras layer
# class ArcMarginProduct(tf.keras.layers.Layer):
#     '''
#     Implements large margin arc distance.

#     Reference:
#         https://arxiv.org/pdf/1801.07698.pdf
#         https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/
#             blob/master/src/modeling/metric_learning.py
#     '''
#     def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
#                  ls_eps=0.0, **kwargs):

#         super(ArcMarginProduct, self).__init__(**kwargs)

#         self.n_classes = n_classes
#         self.s = s
#         self.m = m
#         self.ls_eps = ls_eps
#         self.easy_margin = easy_margin
#         self.cos_m = tf.math.cos(m)
#         self.sin_m = tf.math.sin(m)
#         self.th = tf.math.cos(math.pi - m)
#         self.mm = tf.math.sin(math.pi - m) * m

#     def get_config(self):

#         config = super().get_config().copy()
#         config.update({
#             'n_classes': self.n_classes,
#             's': self.s,
#             'm': self.m,
#             'ls_eps': self.ls_eps,
#             'easy_margin': self.easy_margin,
#         })
#         return config

#     def build(self, input_shape):
#         super(ArcMarginProduct, self).build(input_shape[0])

#         self.W = self.add_weight(
#             name='W',
#             shape=(int(input_shape[0][-1]), self.n_classes),
#             initializer='glorot_uniform',
#             dtype='float32',
#             trainable=True,
#             regularizer=None)

#     def call(self, inputs):
#         X, y = inputs
#         y = tf.cast(y, dtype=tf.int32)
#         cosine = tf.matmul(
#             tf.math.l2_normalize(X, axis=1),
#             tf.math.l2_normalize(self.W, axis=0)
#         )
#         sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
#         phi = cosine * self.cos_m - sine * self.sin_m
#         if self.easy_margin:
#             phi = tf.where(cosine > 0, phi, cosine)
#         else:
#             phi = tf.where(cosine > self.th, phi, cosine - self.mm)
#         one_hot = tf.cast(
#             tf.one_hot(y, depth=self.n_classes),
#             dtype=cosine.dtype
#         )
#         if self.ls_eps > 0:
#             one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes

#         output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
#         output *= self.s
#         return output


# # Return tokens, masks and segments from a text array or series
# def bert_encode(texts, tokenizer, max_len=512):
#     all_tokens = []
#     all_masks = []
#     all_segments = []
    
#     for text in texts:
#         text = tokenizer.tokenize(text)
            
#         text = text[:max_len-2]
#         input_sequence = ["[CLS]"] + text + ["[SEP]"]
#         pad_len = max_len - len(input_sequence)
        
#         tokens = tokenizer.convert_tokens_to_ids(input_sequence)
#         tokens += [0] * pad_len
#         pad_masks = [1] * len(input_sequence) + [0] * pad_len
#         segment_ids = [0] * max_len
        
#         all_tokens.append(tokens)
#         all_masks.append(pad_masks)
#         all_segments.append(segment_ids)
    
#     return np.array(all_tokens), np.array(all_masks), np.array(all_segments)


# # Function to get our text title embeddings using a pre-trained bert model
# def get_text_embeddings(df, max_len = 70):
#     embeds = []
#     module_url = "../input/shopee-external-models/bert_en_uncased_L-24_H-1024_A-16_1"
#     bert_layer = hub.KerasLayer(module_url, trainable = True)
#     vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
#     do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
#     tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)
# #     text = bert_encode(df['title'].values, tokenizer, max_len = max_len)
#     text = bert_encode(df['title_clean'].values, tokenizer, max_len = max_len)
    
#     margin = ArcMarginProduct(
#             n_classes = 11014, 
#             s = 30, 
#             m = 0.5, 
#             name='head/arc_margin', 
#             dtype='float32'
#             )
    
#     input_word_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
#     input_mask = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
#     segment_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
#     label = tf.keras.layers.Input(shape = (), name = 'label')

#     _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
#     clf_output = sequence_output[:, 0, :]
#     x = margin([clf_output, label])
#     output = tf.keras.layers.Softmax(dtype='float32')(x)
#     model = tf.keras.models.Model(inputs = [input_word_ids, input_mask, segment_ids, label], outputs = [output])
    
#     # https://www.kaggle.com/ragnar123/bert-baseline
#     model.load_weights('../input/checkpoints/Bert_123.h5')
#     model = tf.keras.models.Model(inputs = model.input[0:3], outputs = model.layers[-4].output)
#     chunk = 5000
#     iterator = np.arange(np.ceil(len(df) / chunk))
#     for j in iterator:
#         a = int(j * chunk)
#         b = int((j + 1) * chunk)
#         text_chunk = ((text[0][a:b], text[1][a:b], text[2][a:b]))
#         text_embeddings = model.predict(text_chunk, batch_size = 4)
#         embeds.append(text_embeddings)
#     del model
#     text_embeddings = np.concatenate(embeds)
#     print(f'Our text embeddings shape is {text_embeddings.shape}')
#     del embeds
#     gc.collect()
#     return text_embeddings
    

# # Function to get 50 nearest neighbors of each image and apply a distance threshold to maximize cv
# def get_neighbors(df, embeddings, KNN = 50, image = True):
#     model = NearestNeighbors(n_neighbors = KNN)
#     model.fit(embeddings)
#     distances, indices = model.kneighbors(embeddings)
    
#     # Iterate through different thresholds to maximize cv, run this in interactive mode, then replace else clause with a solid threshold
#     predictions = []
#     for k in tqdm(range(embeddings.shape[0])):
#         if image:
#             idx = np.where(distances[k,] < 3.3)[0]
#         else:
#             idx = np.where(distances[k,] < 18.0)[0]
#         ids = indices[k,idx]
#         posting_ids = df['posting_id'].iloc[ids].values
#         predictions.append(posting_ids)
        
#     del model, distances, indices
#     gc.collect()
#     return df, predictions


# # df, df_cu, image_paths = read_dataset()
# text_embeddings = get_text_embeddings(test)
# print('text embeddings2 shape',text_embeddings.shape)
# _ = gc.collect()


# ####################################
# # Get neighbors for text_embeddings
# ####################################
# _knn = 50
# if text_embeddings.shape[0] <= 3:
#     _knn = 3
# else:
#     _knn = 50
    
# test, text_predictions = get_neighbors(test, text_embeddings, KNN=_knn, image=False)
# test['text_embeddings2'] = text_predictions
# del text_embeddings, text_predictions
# gc.collect()


################
# other method
################
# preds = []
# CHUNK = 1024

# print('Finding similar titles...')
# CTS = len(test) // CHUNK
# if len(test) % CHUNK != 0: 
#     CTS += 1

# for j in range( CTS ):
    
#     a = j*CHUNK
#     b = (j+1)*CHUNK
#     b = min(b,len(test))
#     print('chunk',a,'to',b)
    
#     # COSINE SIMILARITY DISTANCE
#     cts = cupy.matmul(text_embeddings, text_embeddings[a:b].T).T
    
#     for k in range(b-a):
#         IDX = cupy.where(cts[k,]>0.7)[0]
#         o = test.iloc[cupy.asnumpy(IDX)].posting_id.values
#         preds.append(o.tolist())
        
# del text_embeddings
# gc.collect()


# test['text_embeddings2'] = preds
# test.head()


# **Phash Feature**

In [ ]:
tmp = test.groupby('image_phash').posting_id.agg('unique').to_dict()
test['phash'] = test.image_phash.map(tmp)
test.head()

# **Image Embedding**

In [ ]:
import argparse
import os

dataset_root = '../input/shopee-product-matching'
dataset_name = 'product'
modelname = 'tf_efficientnet_b2_ns'
test_batch_size = 8
workers = 4
no_cuda = False
seed = 8

MODELS = [
#     '../input/checkpoints/(2021-04-03_172933)product_fold1_512x512_tf_efficientnet_b2_ns_acc(19.50891)_loss(6.97043)_checkpoint27.pth.tar',
#     '../input/checkpoints/(2021-04-08_105923)product_fold3_192x192_dm_nfnet_f0_acc(59.31387)_loss(8.45285)_checkpoint28.pth.tar'
   '../input/checkpoints/(2021-04-09_111137)product_fold3_260x260_tf_efficientnet_b2_ns_acc(0.00000)_loss(0.61341)_checkpoint29.pth.tar'
]

In [ ]:
import cv2
import numpy as np
from PIL import Image

import torch
import torchvision.transforms as transforms
from torchvision.transforms import TenCrop, FiveCrop, ToTensor, Lambda, Normalize

import albumentations as A
from albumentations import ImageOnlyTransform
from albumentations import SmallestMaxSize, HorizontalFlip, Compose, RandomCrop
from albumentations.pytorch import ToTensorV2

from timm.data.transforms_factory import transforms_imagenet_train
from timm.data.random_erasing import RandomErasing

# imagenet
normalize = Normalize(mean=[0.485, 0.456, 0.406],
                      std=[0.229, 0.224, 0.225])


CROP_HEIGHT = 260
CROP_WIDTH = 260


def test_augmentation():
    test_transform = [
        A.Resize(CROP_HEIGHT, CROP_WIDTH),
        A.Normalize(),
        ToTensorV2(),
    ]
    return A.Compose(test_transform)


In [ ]:
import math
from pprint import pprint

import torch
import torch.nn as nn
from torch.nn import functional as F
import torchvision.models as torch_models

import timm


# https://www.kaggle.com/parthdhameliya77/pytorch-resnext50-32x4d-image-tfidf-inference
class ArcMarginProduct(nn.Module):
    def __init__(self, in_features, out_features, scale=30.0, margin=0.50, easy_margin=False, ls_eps=0.0):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.scale = scale
        self.margin = margin
        self.ls_eps = ls_eps  # label smoothing
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(margin)
        self.sin_m = math.sin(margin)
        self.th = math.cos(math.pi - margin)
        self.mm = math.sin(math.pi - margin) * margin

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.scale

        return output
    

class CosineSoftmaxModule(nn.Module):
    def __init__(self, features_dim, nclass=11014):
        super(CosineSoftmaxModule, self).__init__()
        self.nclass = nclass

        in_channels = features_dim

        self.weights = torch.nn.Parameter(torch.randn(in_channels, self.nclass))
        nn.init.xavier_uniform_(self.weight)
        self.scale = torch.nn.Parameter(F.softplus(torch.randn(())))
        nn.init.xavier_uniform_(self.scale)
        self.fc = nn.Linear(in_channels, in_channels)
        self.dropout = nn.Dropout(p=0.1, inplace=False)

    # pooler_output
    def forward(self, x):
        # x = x.view(-1, num_flat_features(x))
        x = self.dropout(x)
        x = self.fc(x)

        features = x
        # Features in rows, normalize axis 1.
        features = F.normalize(features, p=2, dim=1, eps=1e-8)

        # Mean vectors in colums, normalize axis 0.
        weights_normed = F.normalize(self.weights, p=2, dim=0, eps=1e-8)
        logits = self.scale.cuda() * torch.mm(features.cuda(), weights_normed.cuda())  # torch.matmul

        return features, logits


class Model(nn.Module):
    def __init__(self, model_name, use_fc=False, fc_dim=512, nclass=11014):
        super(Model, self).__init__()
        self.model_name = model_name
        self.use_fc = use_fc
        self.nclass = nclass

        print('Building Model Backbone for {} model'.format(model_name))
        model_names = timm.list_models(pretrained=True)
#         pprint(model_names)
        self.backbone = timm.create_model(model_name, pretrained=False, num_classes=nclass)
        # Below code is used when if pretrained is False
        pre_model = torch.load('../input/backbone/tf_efficientnet_b2_ns-00306e48.pth')           
#         print(pre_model)
#         # seresnext50_32x4d
#         del pre_model['fc.weight']
#         del pre_model['fc.bias']
#         # tf_efficientnet_xx_ns
        del pre_model['classifier.weight']
        del pre_model['classifier.bias']
        # dm_nfnet_fx
#         del pre_model['head.fc.weight']
#         del pre_model['head.fc.bias']
        self.backbone.load_state_dict(pre_model, strict=False)

        self.in_channels = 512  # resnet18, resnet34
        if self.model_name in ['resnet18', 'resnet34', 'vgg16', 'vgg19']:
            self.in_channels = 512
        elif self.model_name in ['seresnext50_32x4d', 'resnext101_32x8d', 'resnext50_32x4d',
                               'resnest50d', 'resnest101e', 'resnest200e', 'resnet50',
                               'resnest269e', 'resnet101', 'resnet152']:
            self.in_channels = 2048
        elif self.model_name.startswith('tf_efficientnet_b0'):
            self.in_channels = 1280
        elif self.model_name.startswith('tf_efficientnet_b1'):
            self.in_channels = 1280
        elif self.model_name.startswith('tf_efficientnet_b2'):
            self.in_channels = 1408
        elif self.model_name.startswith('tf_efficientnet_b3'):
            self.in_channels = 1536
        elif self.model_name.startswith('tf_efficientnet_b4'):
            self.in_channels = 1792
        elif self.model_name.startswith('tf_efficientnet_b5'):
            self.in_channels = 2048
        elif self.model_name.startswith('dm_nfnet_f'):
            self.in_channels = 3072

        ##################
        # cosine-softmax
        ##################
        # self.cosine_softmax = CosineSoftmaxModule(self.in_channels, nclass)

        ##################
        # ArcFace - https://www.kaggle.com/parthdhameliya77/pytorch-resnext50-32x4d-image-tfidf-inference
        ##################
        self.margin = ArcMarginProduct(fc_dim, self.nclass)
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.dropout = nn.Dropout(p=0.1, inplace=False)
        self.fc = nn.Linear(self.in_channels, fc_dim)
        self.bn = nn.BatchNorm1d(fc_dim)
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

        
    def forward(self, x):
        batch_size = x.shape[0]
        
        if self.model_name.startswith('tf_efficientnet'):
            x = self.backbone.conv_stem(x)
            x = self.backbone.bn1(x)
            x = self.backbone.act1(x)
            x = self.backbone.blocks(x)
            x = self.backbone.conv_head(x)
            x = self.backbone.bn2(x)
            x = self.backbone.act2(x)
#             x = self.backbone.global_pool(x)
          
        elif self.model_name.startswith('resnet') or \
                self.model_name.startswith('resnext') or \
                self.model_name.startswith('seresnext') or \
                self.model_name.startswith('resnest'):
            x = self.backbone.conv1(x)
            x = self.backbone.bn1(x)
            x = self.backbone.act1(x)
            x = self.backbone.maxpool(x)
            x = self.backbone.layer1(x)
            x = self.backbone.layer2(x)
            x = self.backbone.layer3(x)
            x = self.backbone.layer4(x)
#             x = self.backbone.global_pool(x)
            
        elif self.model_name.startswith('dm_nfnet'):
            x = self.backbone.stem(x)
            x = self.backbone.stages(x)
            x = self.backbone.final_conv(x)
            x = self.backbone.final_act(x)
#             x = self.backbone.head.global_pool(x)
    
        x = self.pooling(x).view(batch_size, -1)

        ##################
        # cosine-softmax
        ##################
        # return self.cosine_softmax(x)

        ##################
        # ArcFace - https://www.kaggle.com/parthdhameliya77/pytorch-resnext50-32x4d-image-tfidf-inference
        ##################
        if self.use_fc:
            x = self.dropout(x)
            x = self.fc(x)
            x = self.bn(x)

        # logits when training
#         return self.margin(x, labels)

        # features when inference
        return x


In [ ]:
import os
import random
import pandas as pd
from PIL import Image

import cv2
import numpy as np
import torch.utils.data as data


class ProductTestDataset(data.Dataset):
    def __init__(self, data_dir, csv, transform=None):
        self.data_dir = data_dir
        self.csv = csv
        self.transform = transform

        df = csv
        self.images = df['image'].values.tolist()
        self.posting_id = df['posting_id'].values.tolist()

    def __getitem__(self, index):
        image_path = os.path.join(self.data_dir, 'test_images', self.images[index])
        # print('#####: ', image_path)

        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform is not None:
            image = self.transform(image=image)['image']

        return image, self.posting_id[index], image_path

    def __len__(self):
        return len(self.images)



In [ ]:
import csv
import os
import time
import random
import gc
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F


NUM_CLASS = 11014


# global variable
best_pred = 0.0
acc_lst_train = []
acc_lst_val = []


cuda = not no_cuda and torch.cuda.is_available()
print(cuda)

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


model = Model(model_name=modelname, use_fc=False)
# print('\n-------------- model details --------------')
# print(model)


if cuda:
    model.cuda()
    model = nn.DataParallel(model)
    

test_df = pd.read_csv("../input/shopee-product-matching/test.csv")

CHUNK = 1024*4
print('Computing image embeddings...')
CTS = len(test_df)//CHUNK

if len(test_df) % CHUNK != 0: 
    CTS += 1

features_pool = []
for resume in MODELS:    
    if resume is not None:
        if os.path.isfile(resume):
            print("=> loading checkpoint '{}'".format(resume))
            checkpoint = torch.load(resume)
            start_epoch = checkpoint['epoch'] + 1
            best_pred = checkpoint['best_pred']
            acc_lst_train = checkpoint['acc_lst_train']
            acc_lst_val = checkpoint['acc_lst_val']
            model.load_state_dict(checkpoint['state_dict'], strict=False)
            print("=> loaded checkpoint '{}' (epoch {})".format(resume, checkpoint['epoch']))
        else:
            raise RuntimeError("=> no resume checkpoint found at '{}'".format(resume))

    model.eval()

    embeds = []    
    for i,j in enumerate(range(CTS)):

        a = j*CHUNK
        b = (j+1)*CHUNK
        b = min(b,len(test_df))
        print('chunk',a,'to',b)

        galleryset = ProductTestDataset(data_dir=dataset_root,
                                        csv=test_df.iloc[a:b],
                                        transform=test_augmentation())

        gallery_loader = torch.utils.data.DataLoader(galleryset, batch_size=1, num_workers=workers)

        gallery_features_list = []
        tbar = tqdm(gallery_loader, desc='\r')
        for batch_idx, (data, pos_id, img_path) in enumerate(tbar):
            if cuda:
                data = data.cuda()

            with torch.no_grad():
                # num_tta = 8
#                 x = torch.stack([data, data.flip(-1), data.flip(-2), data.flip(-1,-2),
#                                  data.transpose(-1,-2), data.transpose(-1,-2).flip(-1),
#                                  data.transpose(-1,-2).flip(-2), data.transpose(-1,-2).flip(-1,-2)], 0)
                # num_tta = 6
                x = torch.stack([data, data.flip(-1), data.flip(-2),
                                 data.transpose(-1,-2), data.transpose(-1,-2).flip(-1),
                                 data.transpose(-1,-2).flip(-2)], 0)

#                 x = torch.stack([data], 0)
                x = x.view(-1, 3, CROP_HEIGHT, CROP_WIDTH)
#                 #################
#                 # cosine-softmax
#                 #################
#                 features, _ = model(x)
                ##########
                # ArcFace
                ##########
                features = model(x)

#                 features = features.view(1, 5, -1).mean(1)
                features = features.view(1, 6, -1).max(1)[0] # values
                gallery_features_list.append(features.cpu().numpy())

        embeds.extend(gallery_features_list)   
    
    features_pool.append(np.concatenate(embeds))
    _ = gc.collect()

# -------------
# max pooling.
# -------------
gallery_features = features_pool[0]
for i in range(1, len(features_pool)):
    gallery_features = np.maximum(gallery_features, features_pool[i])


print('image embeddings shape', gallery_features.shape)


In [ ]:
# # import cudf, cuml
# import cupy

# image_embeddings = cupy.array(gallery_features)

# preds = []
# CHUNK = 1024*4

# print('Finding similar images...')
# CTS = len(image_embeddings) // CHUNK
# if len(image_embeddings) % CHUNK != 0: 
#     CTS += 1

# for j in range( CTS ):
#     a = j*CHUNK
#     b = (j+1)*CHUNK
#     b = min(b,len(image_embeddings))
#     print('chunk',a,'to',b)
    
#     cts = cupy.matmul(image_embeddings, image_embeddings[a:b].T).T
    
#     for k in range(b-a):
#         print('cts[k,]: ', cts[k,])
#         IDX = cupy.where(cts[k,]>0.7)[0]
#         print('IDX: ', IDX)
#         o = test_df.iloc[cupy.asnumpy(IDX)].posting_id.values
# #         preds.append(' '.join(o.tolist()))
#         preds.append(o.tolist())

In [ ]:
if len(test_df) > 3:
    KNN = 50
else : 
    KNN = 3

model = NearestNeighbors(n_neighbors = KNN)
model.fit(gallery_features)
distances, indices = model.kneighbors(gallery_features)
print('distances: ', distances)
print('indices: ', indices)

preds = []
for k in tqdm(range(gallery_features.shape[0])):
    idx = np.where(distances[k,] < 4.8)[0]
    ids = indices[k,idx]
    posting_ids = test_df['posting_id'].iloc[ids].values
    preds.append(posting_ids)

del model, distances, indices
gc.collect()


In [ ]:
test['image_embeddings'] = preds
test.head()

In [ ]:
def combine_for_sub(row):
#     x = np.concatenate([row.text_embeddings, row.text_embeddings2, row.phash, row.image_embeddings])
    x = np.concatenate([row.image_embeddings, row.text_embeddings])
#     print('x: ', x)
#     x = row.image_embeddings
#     print('x: ', x)
#     return ' '.join( np.unique(x) )

    # https://stackoverflow.com/questions/12926898/numpy-unique-without-sort
    indexes = np.unique(x, return_index=True)[1]
    return ' '.join([x[index] for index in sorted(indexes)])


In [ ]:
submit_df = pd.read_csv("../input/shopee-product-matching/sample_submission.csv")

submit_df['matches'] = test.apply(combine_for_sub, axis=1)
submit_df[['posting_id', 'matches']].to_csv('submission.csv', index=False)

sub = pd.read_csv('submission.csv')
sub.head()